In [19]:
# This script is used to collect experiments' results and generate a summary table
# input: path to the directory containing the results
# output: show a summary table and optionally save it to a csv file

In [20]:
output_folder = '../output'

In [ ]:
from pathlib import Path
import pandas as pd
import json
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)
output = Path(output_folder)
items = {
    "Experiment": [],
    "Dataset": [],
    "mIoU": [],
    "mIoU(dcrf)": [],
    "m_TP_T": [],
    "m_TP_P": [],
    "ExperimentPath": [],
}
for experiment in output.iterdir():
    for dataset in experiment.iterdir():
        result_path = dataset / "segmentation_metrics.json"
        dcrf_result_path = dataset / "segmentation_metrics_crf.json"
        items["Experiment"].append(experiment.name)
        items["Dataset"].append(dataset.name)
        items["ExperimentPath"].append(experiment)
        if result_path.exists():
            with open(result_path) as f:
                result = json.load(f)
                items["mIoU"].append(round(result["overall"]["m_IoU"][0],2))
                items["m_TP_T"].append(round(result["overall"]["m_TP_T"][0],2))
                items["m_TP_P"].append(round(result["overall"]["m_TP_P"][0],2))
        else:
            items["mIoU"].append(None)
            items["m_TP_T"].append(None)
            items["m_TP_P"].append(None)
        if dcrf_result_path.exists():
            with open(dcrf_result_path) as f:
                result = json.load(f)
                items["mIoU(dcrf)"].append(round(result["overall"]["m_IoU"][0],2))
        else:
            items["mIoU(dcrf)"].append(None)
table = pd.DataFrame(items)
table